# MNIST Classification using Predictive Coding

This notebook implements a Predictive Coding Network (PCN) for MNIST digit classification.

## Setup and Imports
The following cell sets up our environment with necessary PyTorch imports and checks for GPU availability.


In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
from tqdm.notebook import tqdm

# Check if GPU is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# to display matplotlib plots inline (directly below the cell)
%matplotlib inline


Using device: cpu
